In [5]:
from T13BFLSimpCore import *

In [6]:
dat = 'trips_simpler.csv'
company = [5,4]#[2,3,4,5]
location = [4]#[mk for mk in range(9)]

blockchain = Blockchain()

TypeError: __init__() missing 1 required positional argument: 'prev_hash'

In [ ]:
print("1 - DISTRIBUTION =================================")
import pickle
try:
    locdump = "./models/clis" + str(company) + str(location) + ".p"
    clients = pickle.load(open(locdump, "rb"))
    print("Auto-load from cache")
except:
    print("Generating dist")
    db = Distributor(dat, company, location, settings)
    db.divBoth(settings)
    clients = db.extractClients()
    import pickle
    pickle.dump(clients, open(locdump, "wb"))

1 - DISTRIBUTION =================================
Auto-load from cache


In [ ]:
print("2 - AUTHORIZATION =================================")
print("Using blockchain #" + blockchain.bc_id)
for c in clients:
    c.model = CNNLSTM(settings['frame_in'],settings['frame_out'], epochs=c.epoch)
    blockchain.authorizeClient(c.id)
    c.assignBlockchain(blockchain)
    print(c)

2 - AUTHORIZATION =================================
Using blockchain #0x7f4f8c55fd30
MIN:5/4-5694930,    43917
CLI:4/4--752710,    23207


In [ ]:
print("3 - TRAINING =================================")
# Training period

for c in clients:
    c.model.setData(c.inp_train,c.out_train)
    c.train()
    c.submitModelAsTransaction()

3 - TRAINING =================================
#5/4:1.72s; MIN:5/4 MOD-->CHIDX#  2
#4/4:1.89s; CLI:4/4 MOD-->CHIDX#  2


In [ ]:
print("4 - MINING =================================")
for c in clients:
    if type(c) == Miner:
        c.mine()

4 - MINING =================================
Start mining: 5/4 | PREV={} CURR={'5694930588976327424': 0.6254221064605102, '-7527100985386906068': 0.6695347673049343}
BLK: C:0x7f4f8#  2 <-- PH=1      , PF=5694930, RL: QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH
